# Library

In [108]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm

from gensim.models import Word2Vec, KeyedVectors

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Word2Vec

paper: [Efficient Estimation of Word Representations in
Vector Space](https://arxiv.org/pdf/1301.3781) <br>
reference: [Word2Vec](https://wikidocs.net/22660) <br>
site: https://word2vec.kr/ <br>

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=5, workers=8, sg=0)   # train
> model.wv.most_similar('[word]')   # 입력 단어와 가장 유사한 단어 출력
>
> model.wv.save_word2vec_format('[filename]')   # save
> model = KeyedVectors.load_word2vec_format('[filename]')   # load
> ```

<br>

parameters
- vector_size: vector dimension (몇 차원 벡터로 학습시킬 것인가)
- window: window size
- min_count: 학습 시 학습할 단어가 최소 몇 개 등장해야 학습할 것인가
- workers: 학습에 사용할 cpu 수
- sg
    - 0: CBOW
    - 1: Skip-gram

## Implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class CBOWWord2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.output = nn.Linear(self.embedding_dim, self.vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.sum(axis=1)
        output = self.output(x)

        return output        

In [6]:
# x = torch.zeros((5, 5), dtype=torch.long)
# for i in range(len(x)):
#     x[i, i] = 1
# input_ = torch.cat([x[:2], x[3:]])
input_ = torch.Tensor([[0, 1, 3, 4]]).long()
word2vec = CBOWWord2Vec(5, 16)
logit = word2vec(input_)

In [10]:
F.cross_entropy(logit, torch.Tensor([2000]).long())

tensor(2.5080, grad_fn=<NllLossBackward0>)

# Embedding Projector

paper: [Embedding Projector: Interactive Visualization and
Interpretation of Embeddings](https://arxiv.org/pdf/1611.05469v1.pdf) <br>
site: https://projector.tensorflow.org/ <br>

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```cmd
> !python -m gensim.scripts.word2vec2tensor --input [model_name] --output [output_model_name]
> ```



# Practice

## english news

In [110]:
data = pd.read_csv('./data/abcnews-date-text.csv')

# 각 기사 제목(headline_text)에 대해 단어를 토큰화하여 'tokens' 컬럼에 저장
data['tokens'] = data.headline_text.apply(lambda x: nltk.wordpunct_tokenize(x))

# NLTK의 영어 불용어(stop words) 리스트를 로드
stop_words = stopwords.words('english')

# 토큰 중 불용어를 제거하여 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [token for token in tokens if token not in stop_words])

# 토큰들을 명사 형태로 표제어 추출(lemmatization)한 후, 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [
    WordNetLemmatizer().lemmatize(token) for token in tokens if token not in stop_words
])

# 토큰들을 동사 형태로 표제어 추출(lemmatization)한 후, 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [
    WordNetLemmatizer().lemmatize(token, pos='v') for token in tokens if token not in stop_words
])

# 길이가 3자 이상인 토큰만 남기고 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [token for token in tokens if len(token) >= 3])


In [111]:
model = Word2Vec(
    sentences=data.tokens.tolist(),
    vector_size=128,
    window=5,
    min_count=5,
    sg=1,
)

In [112]:
model.wv.save_word2vec_format('./word2vec.txt')

In [18]:
# trump와 유사한 단어 탐색

model.wv.most_similar('trump')

[('donald', 0.8843256831169128),
 ('obama', 0.7616173028945923),
 ('clinton', 0.7524031400680542),
 ('obamas', 0.6814157366752625),
 ('romney', 0.666195809841156),
 ('merkel', 0.6616398096084595),
 ('republican', 0.6558955907821655),
 ('barack', 0.6552227735519409),
 ('melania', 0.6508939266204834),
 ('gop', 0.6417585611343384)]

In [21]:
# 학습되지 않은 token은 vocab에 없어 key error 발생

model.wv.most_similar('pneumonoultramicroscopicilisocovlcanocniosis')

KeyError: "Key 'pneumonoultramicroscopicilisocovlcanocniosis' not present in vocabulary"

## 기자회견

In [35]:
comments = pd.read_pickle('./data/comments_minheejin.pickle')

In [36]:
kiwi = Kiwi()
# 단어 사전에 단어 추가
kiwi.add_user_word('어도어', 'NNP')
kiwi.add_user_word('빌리프랩', 'NNP')
kiwi.add_user_word('빌리프렙', 'NNP')
kiwi.add_user_word('아일리스', 'NNP')
kiwi.add_user_word('르세라핌', 'NNP')
kiwi.add_user_word('피프티피프티', 'NNP')
kiwi.add_user_word('뉴진스', 'NNP')
kiwi.add_user_word('아일릿', 'NNP')
kiwi.add_user_word('하이브', 'NNP')
kiwi.add_user_word('방시혁', 'NNP')
kiwi.add_user_word('힛뱅맨', 'NNP')
kiwi.add_user_word('힛맨뱅', 'NNP')
kiwi.add_user_word('민희진', 'NNP')
kiwi.add_user_word('미니진', 'NNP')
kiwi.add_user_word('희진', 'NNP')
kiwi.add_user_word('진짜사나이이', 'NNP')
kiwi.add_user_word('레퍼런스', 'NNG')
kiwi.add_user_word('언플', 'NNG')
kiwi.add_user_word('대퓨', 'NNG')
kiwi.add_user_word('대퓨님', 'NNG')
kiwi.add_user_word('개저씨', 'NNG')
kiwi.add_user_word('댓글부대', 'NNG')

# 불용어 사전에 불용어 추가
stopwords = Stopwords()
stopwords.add(('웩퉥', 'NNG'))
stopwords.add(('결국', 'NNG'))

def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

# comments 데이터프레임의 'textOriginal' 열에 있는 텍스트에 대해
# extract_tokens 함수를 적용하여 추출된 토큰들을 'tokens' 열에 저장
comments['tokens'] = comments.textOriginal.apply(lambda x: extract_tokens(x, kiwi, stopwords))

In [50]:
model = Word2Vec(
    sentences=comments.tokens.tolist(),
    vector_size=16,
    window=1,
    min_count=3,
    sg=1,
)

In [57]:
model.wv.most_similar('민희진')

[('레이블', 0.9915684461593628),
 ('방시혁', 0.9895018935203552),
 ('상장', 0.988251805305481),
 ('주식', 0.9861562848091125),
 ('몰이', 0.9855770468711853),
 ('자본', 0.9853259921073914),
 ('박지원', 0.9847903251647949),
 ('이단', 0.9847900867462158),
 ('소속사', 0.9844740629196167),
 ('변호사', 0.9844532608985901)]

In [ ]:
comments['tokens'].apply(len).describe()

In [63]:
model.wv['인기']

array([-0.19295599, -0.5011149 ,  0.18954143,  0.4211095 ,  0.41131747,
        0.22440618,  0.62413543, -0.11859904, -0.13695215,  0.05069742,
       -0.03767992, -0.34668222,  0.02807992, -0.4721375 , -0.06153082,
        0.08143295], dtype=float32)

In [72]:
comments['vector_mean'] = comments.tokens.apply(
    lambda x: np.mean(
        [model.wv[word] for word in x if len(x) >= 1 and word in model.wv.index_to_key]
    , axis=0)
)

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## News

In [93]:
news = []
file_paths = glob.glob('./data/news_data/**/*.txt', recursive=True)
for file_path in tqdm(file_paths):
    with open(file_path) as file:
        temp = file.read()
        news.append(temp)

100%|██████████| 1600/1600 [00:00<00:00, 13051.32it/s]


In [95]:
def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

In [96]:
kiwi = Kiwi()
stopwords = Stopwords()

news = pd.DataFrame(news, columns=['contents'])
news = news.drop_duplicates()
news.contents = news.contents.str.replace('\s{1,}', ' ', regex=True)
news['tokens'] = news.contents.apply(lambda x: extract_tokens(x, kiwi, stopwords))

In [99]:
model = Word2Vec(
    sentences=news.tokens.tolist(),
    vector_size=64,
    window=3,
    min_count=5,
    sg=1,
)

In [100]:
model.wv.most_similar('삼성전자')

[('LG전자', 0.9372370839118958),
 ('애플', 0.917360246181488),
 ('갤럭시S', 0.9171818494796753),
 ('갤럭시', 0.9132965207099915),
 ('출고', 0.9128235578536987),
 ('씽큐', 0.9080085754394531),
 ('스마트폰', 0.8961372971534729),
 ('인하', 0.8925694823265076),
 ('하반기', 0.8884367346763611),
 ('수요', 0.8881014585494995)]

In [105]:
model.wv.save_word2vec_format('./word2vec.txt')